# 数学优化建模简介

## 目标和先决条件

该建模示例的目的是在混合整数规划（MIP）问题的公式化中介绍关键元素。对于MIP问题公式的每个组成部分，我们提供描述，相关的Python代码以及描述该组成部分的数学符号。

为了完全理解此部分的内容，读者应：


* 熟悉Python。
* 有相关学科背景，包括但不限于工程、计算机科学、经济学、统计学等各类“硬核”科学，或包含定量模型和方法的任一学科。

读者还应查询 Gurobi Python API 的相关 [文档](https://www.gurobi.com/resources/?category-filter=documentation)。此外，在《混合整数线性编程的系列教程》视频中详细介绍了此文档的内容。

你可以在 [这里](https://www.gurobi.com/resource/tutorial-mixed-integer-linear-programming/) 观看视频教程

**注意：** 你可以通过单击 [此处](https://github.com/arvinxx/gurobi-and-mathematical-modeling/archive/master.zip) 下载包含此示例和其他示例的代码仓库。为了正确运行此 Jupyter Notebook，您必须具有 Gurobi 许可证。如果您没有，则可以 *商业用户身份* 申请[评估许可证](https://www.gurobi.com/downloads/request-an-evaluation-license/)，或以 *学术用户* 的身份下载 [免费许可证](https://www.gurobi.com/academia/academic-program-and-licenses)。

## 问题描述

假如有一家咨询公司，它有三个空缺职位：测试、Java程序员和架构师。这三个职位的最佳候选人（资源）是：Carlos、Joe 和 Monika。
这家咨询公司对每个候选人进行了能力测试，以评估他们完成每项工作的能力。这些测试的结果称为 「匹配分数」（matching scores）。假设一个职位只能分配一个应聘者，并且最多可以为一个应聘者分配一个职位。

我们的目标是：确定候选人和工作岗位的分配，以使每个工作岗位都得到满足。每个候选人最多分配一个岗位，分配后的总「匹配得分」最大。


## 数学优化

数学优化（也称为数学编程）是一种声明性方法，其中建模者制定了一个优化问题，该问题捕获了复杂决策问题的关键特征。然后 Gurobi 使用最先进的数学和计算机科学技术来解决这一类数学优化问题。

数学优化模型具有五个组成部分：

* 数据集（Sets）
* 参数(Parameters)
* 决策变量(Decision variables)
* 约束（Constraints）
* 目标函数（Objective function(s)）


以下 Python 代码导入了 Gurobi 模块，并将 `GRB` 类导入了主命名空间。


In [78]:
import gurobipy as gp
from gurobipy import GRB

## 资源分配问题

### Data

列表 $R$ 包含三个资源的名称：Carlos，Joe 和 Monika。
列表 $J$ 包含工作职位的名称：Tester（测试），Java Developer（Java开发） 和 Architect（架构）。

$r \in R$：索引和资源集。资源 $r$ 属于资源集 $R$。

$j \in j$：索引和岗位集。作业 $j$ 属于岗位集 $J$。






In [79]:
# 资源集和岗位集
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

以下匹配得分表中列出了每种资源参与每种工作岗位的能力：

![scores](https://gw.alipayobjects.com/zos/antfincdn/NWZW0Xv%24Ms/1d4ac105-34ee-446b-8974-77cdc9410763.png)

对于每个资源 $r$ 和作业 $j$，都有一个对应的匹配得分 $s$。匹配的分数 $s$ 只能接受0到100之间的值。

也就是说，对于 $r \in R $ 和 $j \in J$ ，都有 $s_{r,j} \in [0,100]$ 。

我们使用 Gurobi Python 的`multidict` 函数来初始化一个或多个字典。该函数以字典为参数。键代表资源和工作岗位的可能组合。

In [80]:
# 匹配分数数据
combinations, scores = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

以下构造函数创建一个空的 Model 对象 `m` 。我们通过传递字符串`"RAP"` 作为参数来指定模型名称。 Model 对象 `m` 包含一个优化问题。该优化问题由一组决策变量，一组约束和目标函数组成。

In [81]:
# 声明并初始化模型
m = gp.Model('RAP')

## 决策变量

为了解决此分配问题，我们需要确定将哪个资源分配给哪个工作。我们为工作的每种可能资源分配引入一个决策变量。因此，我们有 9 个决策变量。

为了简化模型公式的数学符号，我们定义了以下资源和工作的指标:

![variables](https://gw.alipayobjects.com/zos/antfincdn/LEzDic7PJB/81d8f5f2-7a92-49c4-b3a0-11879715e2fd.png)

例如，$x_{2,1}$ 是与将资源 Joe 分配给工作岗位 Tester 的决策变量。因此，如果$r \in R$ 被分配给 $j \in J$，则决策变量 $x_{r,j}$ 等于1，否则为0。

`Model.addVars()` 方法为 Model 对象创建决策变量。此方法返回包含新创建的决策变量。该变量类型为 Gurobi 专有的 `tupledict` 对象。我们将 `combinations` 对象作为指定变量索引的第一个参数。`name` 关键字用于为新创建的决策变量命名。默认情况下，变量被假定为非负数。


In [82]:
# 为RAP模型创建决策变量
x = m.addVars(combinations, name="assign")

## 职位限制

现在我们讨论与职位相关的约束。这些约束条件需要确保每个职位都恰好由一个候选人填充。

Tester 职位的约束要求将资源1（Carlos），资源2（Joe）或资源3（Monika）分配给该 职位。这对应于以下约束。

约束 (Tester=1)

$$
x_{1,1} + x_{2,1} + x_{3,1} = 1
$$

类似地，Java职位和架构师职位的约束可以定义如下。

约束 (Java Developer = 2)

$$
x_{1,2} + x_{2,2} + x_{3,2} = 1
$$

约束 (Architect = 3)

$$
x_{1,3} + x_{2,3} + x_{3,3} = 1
$$

职位约束由下表的列定义。

![jobs](https://gw.alipayobjects.com/zos/antfincdn/wCDVAxhuEU/642de4a4-eae6-496b-8237-98a3acebf283.png)

通常情况下，可以将 Tester 职位的约束定义如下。

$$
x_{1,1} + x_{2,1} + x_{3,1} = \sum_{r=1}^{3 } x_{r,1} =  \sum_{r \in R} x_{r,1} = 1
$$
可以以类似简洁的方式定义所有职位约束。对于$j \in J$，取所有候选人的决策变量的总和。我们可以如下编写相应的职位约束。

$$
\sum_{r \in R} x_{r,j} = 1
$$

Gurobi 的 `Model.addConstrs()` 方法定义了 Model 对象 `m` 的工作约束。此方法返回作业约束的变量，该变量类型也为 `tupledict`。


In [83]:
# 创建工作约束
jobs = m.addConstrs((x.sum('*',j) == 1 for j in J), name='job')

此方法的第一个参数`x.sum(‘*’, j)`是 sum 方法，它定义作业约束的 LHS 如下：

对于作业 $J$ 中的每个作业 $j$ ，取所有资源上决策变量的总和。`==` 定义了一个相等约束，数字 1 是这些约束的RHS。

这些约束意味着每个作业只能分配一个资源。第二个参数是此类约束的名称。

## 资源约束


资源约束需要确保至多为每个资源分配一个作业。也就是说，可能不是所有的资源都被分配了。
比如这样约束：Carlos 最多分配一个工作: 工作1(Tester)、工作2 (Java)或工作3(架构师)。这个约束如下书写：

约束 (Carlos=1)

$$
x_{1, 1} + x_{1, 2} + x_{1, 3}  \leq 1.
$$

这个约束使它们的值小于或等于1，以允许 Carlos 不被分配到任何工作的可能性。同样，对资源 Joe 和 Monika 的约束可以定义如下:

约束 (Joe=2)

$$
x_{2, 1} + x_{2, 2} + x_{2, 3}  \leq 1.
$$

约束 (Monika=3)

$$
x_{3, 1} + x_{3, 2} + x_{3, 3}  \leq 1.
$$

请注意，资源约束是由下表的行定义的。

![resources](https://gw.alipayobjects.com/zos/antfincdn/Kaf%26AdwmYU/0dd89ba0-5688-4618-b3dd-4c26571e976f.png)

资源 Carlos 的约束可以定义如下。

$$
x_{1, 1} + x_{1, 2} + x_{1, 3} = \sum_{j=1}^{3 } x_{1,j} = \sum_{j \in J} x_{1,j} \leq 1.
$$

同样，每个约束都可以用简洁的方式书写。对于$r \in R$，对所有作业的决策变量求和。我们可以将相应的资源约束写如下。

$$
\sum_{j \in J} x_{r,j} \leq  1.
$$

Gurbo 的 `Model.addConstrs()` 方法定义了 Model 对象 `m` 的资源约束。
此方法的第一个参数 `x.sum(r, ‘*’)` 是求和方法，它定义资源约束的LHS如下： 对于资源集 $R$中的每个资源$r$，对所有工作进行决策变量的求和。

`<=`定义了一个更少或相等的约束，约束的RHS是 1。

这些限制意味着每个资源最多只能分配1个作业。

第二个参数是此类约束的名称。


In [84]:
# 创建资源约束
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')

## 目标函数

目标函数是使满足作业和资源约束的作业的总匹配得分最大化。

对于测试人员作业，如果分配了资源Carlos，其匹配分数为 $53x_{1,1}$, 如果分配了资源 Joe，匹配分数为 $80x_{2,1}$, 如果分配了资源 Monika，匹配分数为  $53x_{3,1}$。

因此，Tester作业的匹配分数如下，其中该总和中的决策变量只有一项值为 1。

$$
53x_{1,1} + 80x_{2,1} + 53x_{3,1}. 
$$

类似地，Java开发人员和架构师工作的匹配分数定义如下。Java开发人员作业的匹配分数是:

$$
27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}.
$$

架构师工作的匹配分数是:

$$
13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}.
$$

总匹配得分为下表中每个单元格的总和。

![objfcn](https://gw.alipayobjects.com/zos/antfincdn/irHnhJepQ8/188b942e-a9d6-4248-8cfb-54accd1c84fb.png)

目标是使分配的总匹配分数最大化。因此，目标函数定义如下。

\begin{equation}
\text{Maximize} \quad (53x_{1,1} + 80x_{2,1} + 53x_{3,1}) \; +
\end{equation}

\begin{equation}
\quad (27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}) \; +
\end{equation}

\begin{equation}
\quad (13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}).
\end{equation}

目标函数中括号中的每一项可以表示为:

\begin{equation}
(53x_{1,1} + 80x_{2,1} + 53x_{3,1}) = \sum_{r \in R} s_{r,1}x_{r,1}.
\end{equation}

\begin{equation}
(27x_{1, 2} + 47x_{2, 2} + 73x_{3, 2}) = \sum_{r \in R} s_{r,2}x_{r,2}.
\end{equation}

\begin{equation}
(13x_{1, 3} + 67x_{2, 3} + 47x_{3, 3}) = \sum_{r \in R} s_{r,3}x_{r,3}.
\end{equation}

因此，目标函数可以简写为：

\begin{equation}
\text{Maximize} \quad \sum_{j \in J} \sum_{r \in R} s_{r,j}x_{r,j}.
\end{equation}

Gurobi 的 `Model.setobobjective()` 方法定义了 Model 对象 `m` 的目标函数。

In [85]:
# 目标:使所有作业的总匹配分数最大化
m.setObjective(x.prod(scores), GRB.MAXIMIZE)

第一个参数指定了目标函数的表达式。

> 注意：匹配的分数参数 score 和分配决策变量 x 都是在 `combinations` 的键上定义的。因此，我们可以使用`x.prod(score)` 的方式,得到 score 矩阵与 x 变量矩阵的元素乘法的和。

第二个参数`GRB.MAXIMIZE`， `MAXIMIZE`，是优化核心目标。在这种情况下，我们想要 **最大化** 所有分配的总匹配分数。

我们使用 Gurobi 的 `write()` 方法将模型公式写入 `RAP.lp` 文件。

In [86]:
# 保存模型以供查看
m.write('RAP.lp')

![RAP](https://gw.alipayobjects.com/zos/antfincdn/J6j1Hm9Kis/8afcfa53-9d62-462d-a228-de904d9c72ed.png)

我们使用 Gurobi 的 `optimize()` 方法来为模型对象 `m` 定义的问题求解。

In [87]:
# 运行优化引擎
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb6602fb2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  1.930000000e+02


使用 Gurobi 的`Model.getVars()` 方法来检索模型对象 `m` 中所有变量。

`.x` 属性用于查询当前可行解下的变量值，`.varName` 属性用于获得决策变量的名称。

In [88]:
# 显示决策变量的值
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# 显示最佳总匹配分数
print('匹配得分总分: ', m.objVal)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
匹配得分总分:  193.0


最佳分配方案是：

* Carlos -> Tester, 匹配得分为 53
* Joe -> Architect, 匹配得分为 of 67
* Monika ->  Java Developer, 匹配得分为 73.

最大匹配得分的总分为193分。


## 具有预算约束的资源分配问题

Now, assume there is a fixed cost $C_{r,j}$ associated with assigning a resource $r \in R$ to job $j \in J$. Assume also that there is a limited budget $B$ that can be used for job assignments.

The cost of assigning Carlos, Joe, or Monika to any of the jobs is $\$1,000$ , $\$2,000$ , and $\$3,000$  respectively. The available budget is $\$5,000$.

### Data

The list $R$ contains the names of the three resources: Carlos, Joe, and Monika.
The list $J$ contains the names of the job positions: Tester, Java Developer, and Architect.

The Gurobi Python ``multidict`` function initialize two dictionaries: 
* "scores" defines the matching scores for each resource and job combination.
* "costs" defines the fixed cost associated of assigning a resource to a job.



In [89]:
# Resource and job sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

# Matching score data
# Cost is given in thousands of dollars
combinations, scores, costs = gp.multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'): [27, 1],
    ('Carlos', 'Architect'): [13,1],
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'): [47, 2],
    ('Joe', 'Architect'): [67, 2],
    ('Monika', 'Tester'): [53, 3] ,
    ('Monika', 'JavaDeveloper'): [73, 3],
    ('Monika', 'Architect'): [47, 3]
})

# Available budget (thousands of dollars)
budget = 5

The following constructor creates an empty ``Model`` object “m”. The ``Model`` object “m” holds a single optimization problem. It consists of a set of variables, a set of constraints, and the objective function.

In [90]:
# Declare and initialize model
m = gp.Model('RAP2')

### Decision variables

The decision variable $x_{r,j}$ is 1 if $r \in R$ is assigned to job $j \in J$, and 0 otherwise.

The ``Model.addVars()`` method defines the decision variables for the model object “m”.  

Because there is a budget constraint, it is possible that not all of the jobs will be filled. To account for this, we define a new decision variable that indicates whether or not a job is filled.

Let $g_{j}$ be equal 1 if job $j \in J$ is not filled, and 0 otherwise. This variable is a gap variable that indicates that a job cannot be filled.

***Remark:*** For the previous formulation of the RAP, we defined the assignment variables as non-negative and continuous which is the default value of the ``vtype`` argument of the ``Model.addVars()`` method.
However, in this extension of the RAP, because of the budget constraint we added to the model, we need to explicitly define these variables as binary. The ``vtype=GRB.BINARY`` argument of the ``Model.addVars()`` method defines the assignment variables as binary.

In [91]:
# Create decision variables for the RAP model
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

# Create gap variables for the RAP model
g = m.addVars(J, name="gap")

### Job constraints

Since we have a limited budget to assign resources to jobs, it is possible that not all the jobs can be filled. For the job constraints, there are two possibilities either a resource is assigned to fill the job, or this job cannot be filled and we need to declare a gap. This latter possibility is captured by the decision variable $g_j$. Therefore, the job constraints are written as follows. 

For each job $j \in J$, exactly one resource must be assigned to the job, or the corresponding $g_j$ variable must be set to 1:

$$
\sum_{r \: \in \: R} x_{r,\; j} + g_{j} = 1.
$$


In [92]:
# Create job constraints
jobs = m.addConstrs((x.sum('*',j) + g[j]  == 1 for j in J), name='job')

### Resource constraints

The constraints for the resources need to ensure that at most one job is assigned to each resource. That is, it is possible that not all the resources are assigned. Therefore, the resource constraints are written as follows.

For each resource $r \in R$, at most one job can be assigned to the resource:

$$
\sum_{j \: \in \: J} x_{r,\; j} \leq 1.
$$

In [93]:
# Create resource constraints
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')

### Budget constraint

This constraint ensures that the cost of assigning resources to fill job requirements do not exceed the budget available. The costs of assignment and budget are in thousands of dollars.

The cost of filling the Tester job is $1x_{1,1}$, if resource Carlos is assigned, or $2x_{2,1}$, if resource Joe is assigned, or $3x_{3,1}$, if resource Monika is assigned.
Consequently, the cost of filling the Tester job is as follows, where at most one term in this summation will be nonzero.

$$
1x_{1,1} + 2x_{2,1} + 3x_{3,1}. 
$$

Similarly, the cost of filling the Java Developer and Architect jobs are defined as follows. The cost of filling the Java Developer job is:

$$
1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}.
$$

The cost of filling the Architect job is:

$$
1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}.
$$

Hence, the total cost of filling the jobs should be less or equal than the budget available.

\begin{equation}
(1x_{1,1} + 2x_{2,1} + 3x_{3,1}) \; +
\end{equation}

\begin{equation}
(1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}) \; +
\end{equation}

\begin{equation}
(1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}) \leq 5
\end{equation}

Each term in parenthesis in the budget constraint can be expressed as follows.

\begin{equation}
(1x_{1,1} + 2x_{2,1} + 3x_{3,1}) = \sum_{r \in R} C_{r,1}x_{r,1}.
\end{equation}

\begin{equation}
(1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}) = \sum_{r \in R} C_{r,2}x_{r,2}.
\end{equation}

\begin{equation}
(1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}) = \sum_{r \in R} C_{r,3}x_{r,3}.
\end{equation}

Therefore, the budget constraint can be concisely written as:

\begin{equation}
\sum_{j \in J} \sum_{r \in R} C_{r,j}x_{r,j} \leq B.
\end{equation}

The ``Model.addConstr()`` method of the Gurobi/Python API defines the budget constraint of the ``Model`` object “m”. 
The first argument of this method, "x.prod(costs)", is the prod method and defines the LHS of the budget constraint. The $<=$ defines a less or equal constraint, and the budget amount available is the RHS of the constraint.
This constraint is saying that the total cost of assigning resources to fill jobs requirements cannot exceed the budget available.
The second argument is the name of this constraint.

In [94]:
budget = m.addConstr((x.prod(costs) <= budget), name='budget')

## Objective function

The objective function is similar to the RAP. The first term in the objective is the total matching score of the assignments. In this extension of the RAP, it is possible that not all jobs are filled; however, we want to heavily penalize this possibility. For this purpose, we have a second term in the objective function that takes the summation of the gap variables over all the jobs and multiply it by a big penalty $M$.

Observe that the maximum value of a matching score is 100, and the value that we give to $M$ is 101. The rationale behind the value of $M$ is that having gaps heavily deteriorates the total matching scores value.

Consequently, the objective function is to maximize the total matching score of the assignments minus the penalty associated of having gap variables with a value equal to 1.

$$
\max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} s_{r,j}x_{r,j} -M \sum_{j \in J} g_{j}
$$

In [95]:
# Penalty for not filling a job position
M = 101

In [96]:
# Objective: maximize total matching score of assignments
# Unfilled jobs are heavily penalized
m.setObjective(x.prod(scores) - M*g.sum(), GRB.MAXIMIZE)

In [97]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 7 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xf3c6f8c8
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.00000   160%     -    0s
     0     0  121.66667    0    7   52.00000  121.66667   134%     -    0s
     0     0     cutoff    0      

The definition of the objective function includes the penalty of no filling jobs. However, we are interested in the optimal total matching score value when not all the jobs are filled. For this purpose, we need to compute the total matching score value using the matching score values $s_{r,j}$ and the assignment decision variables $x_{r,j}$.

In [98]:
# Compute total matching score from assignment variables
total_matching_score = 0
for r, j in combinations:
    if x[r, j].x > 1e-6:
        print(x[r, j].varName, x[r, j].x) 
        total_matching_score += scores[r, j]*x[r, j].x

print('Total matching score: ', total_matching_score)  

assign[Joe,Tester] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  153.0


### 分析

Recall that the budget is $\$5,000$, and the total  cost associated of allocating the three resources is $\$6,000$. This means that there is not enough budget to allocate the three resources we have. Consequently, the Gurobi Optimizer must choose two resources to fill the jobs demand, leave one job unfilled, and maximize the total matching scores. Notice that the two top matching scores are 80% (Joe for the Tester job) and 73% (Monika for the Java Developer job). Also, notice that the lowest score is 13% (Carlos for the Architect job). Assigning Joe to the Tester job, Monika to the Java Developer job, and nobody to the Architect job costs $\$5,000$  and yields a total matching score of 153. This is the optimal solution found by the Gurobi Optimizer.

Copyright © 2020 Gurobi Optimization, LLC

翻译 By Arvin Xu